In [1]:
import sys, os
sys.path.append('/Users/paulos/Documents/work/diplo')
os.environ['DJANGO_SETTINGS_MODULE'] = 'diplo.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()

from data.models import *

In [2]:
import pandas as pd


In [8]:
wb = pd.ExcelFile(u"Base Projeto BID - Fluxos (Matriz) - com cortes.xlsx")

IOError: [Errno 2] No such file or directory: u'etl/Base Projeto BID - Fluxos (Matriz) - com cortes.xlsx'

In [4]:
planilhas = wb.sheet_names
planilhas

[u'SIHTotal - Total',
 u'SIHTotal - Cl\xednica M\xe9dica',
 u'SIHTotal - Cl\xednica Pedi\xe1trica',
 u'SIHTotal - Cl\xednica Obst\xe9trica',
 u'SIHTotal - Cl\xednica Cir\xfarg Ped',
 u'SIHTotal - Cl\xednica Cir\xfarg Ad',
 u'SIHTotal - Psiquiatria',
 u'SIHTotal - Cr\xf4nicos',
 u'SIHTotal - Di\xe1rias de UTI',
 u'SIHM\xe9dComp - Total',
 u'SIHM\xe9dComp - Cl\xednica M\xe9dica',
 u'SIHM\xe9dComp - Cl\xednica Pedi\xe1trica',
 u'SIHM\xe9dComp - Cl\xednica Obst\xe9trica',
 u'SIHM\xe9dComp - Cl\xednica Cir\xfarg Ped',
 u'SIHM\xe9dComp - Cl\xednica Cir\xfarg Ad',
 u'SIHM\xe9dComp - Psiquiatria',
 u'SIHM\xe9dComp - Cr\xf4nicos',
 u'SIHM\xe9dComp - Di\xe1rias de UTI',
 u'SIHAltComp - Total',
 u'SIHAltComp - Cl\xednica M\xe9dica',
 u'SIHAltComp - Cl\xednica Pedi\xe1trica',
 u'SIHAltComp - Cl\xednica Cir\xfarg Ped',
 u'SIHAltComp - Cl\xednica Cir\xfarg Ad',
 u'SIHAltComp - Di\xe1rias de UTI']

In [5]:
def importa():
    # hardcoded com os codigos do banco
    categorias = {
        u'SIHTotal': 66,
        u'SIHMédComp': 67,
        u'SIHAltComp': 68,    
    }
    for cat in Categoria.objects.filter(pk__in=categorias.values()):
        cat.indicadorfluxo_set.all().delete()
    for planilha in planilhas:
        df = wb.parse(planilha)
        print planilha
        i = list(df.columns).index(u'Nome da região de saúde (fevereiro de 2015) - BID')
        df = df[[_ for _ in df.columns if _ not in df.columns[i+1:-63]]]
        cat, nom = planilha.split(' - ')
        cat = Categoria.objects.get(pk=categorias[cat])
        localidades = Localidade.objects.filter(tipo='regsau')
        ind = IndicadorFluxo.objects.create(
            categoria=cat,
            nome=nom,         
        )
        for n, row in df.iterrows():
            origem=Localidade.objects.get(codigo=row[u'Código da região de saúde (fevereiro de 2015) - BID'])
            for loc in localidades:
                if loc.nome + '.1' not in row:
                    print u"Coluna não encontrada para localidade {}".format(loc)
                col = row[loc.nome + '.1']
                DadoFluxo.objects.create(
                    indicador=ind, 
                    origem=origem,
                    destino=loc,
                    ano=2016, 
                    valor=col,
                )

In [6]:
importa()

SIHTotal - Total


KeyboardInterrupt: 

In [ ]:
df2 = df1[[_ for _ in df1.columns if _ not in df1.columns[4:-63]]]


In [ ]:
df2

In [ ]:
sih = Categoria.objects.get(pk=66)

In [ ]:
i = IndicadorFluxo.objects.create(categoria=sih, nome='Total')

In [ ]:
cols = dict([(col + '.1', cod) for col, cod in df1[[2,1]].values.tolist()])

In [ ]:
for n, row in df2.iterrows():
    for col in cols:
        DadoFluxo.objects.create(indicador=i, origem=Localidade.objects.get(codigo=row[1]), destino=Localidade.objects.get(codigo=cols[col]), ano=2015, valor=row[col])

In [ ]:
row[1]

In [ ]:
Localidade.objects.get(codigo=35174)